**NEW**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    device_map="cpu"
)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0


In [ ]:
from transformers import pipeline as hf_pipeline
from langchain_core.prompts import ChatPromptTemplate

class SentimentChatbot:
    def __init__(self, llm):
        self.history = []
        self.llm = llm
        self.sentiment = hf_pipeline("sentiment-analysis")

    def analyze_sentiment(self, text):
        out = self.sentiment(text)[0]
        label = out["label"]
        score = out["score"]

        # Normalize to EXACT output: Positive / Negative
        if label.upper() == "NEGATIVE":
            final_label = "Negative"
        else:
            final_label = "Positive"

        return final_label, score

    def reply(self, user_msg):
        # 1️⃣ Save user message
        self.history.append({"role": "user", "text": user_msg})

        # 2️⃣ SENTIMENT FIRST
        sentiment_label, sentiment_score = self.analyze_sentiment(user_msg)

        print(f"Sentiment: {sentiment_label}")

        # 3️⃣ Generate chatbot reply
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful chatbot."),
            ("human", "{message}")
        ])

        final_prompt = prompt.format_messages(message=user_msg)

        response = self.llm.invoke(final_prompt)

        # If LangChain returns list instead of string (rare)
        if isinstance(response, list):
            response = response[0]["generated_text"]

        # 4️⃣ Save bot response
        self.history.append({"role": "assistant", "text": response})

        return response

    def message_level_sentiment(self):
        results = []
        for msg in self.history:
            if msg["role"] == "user":
                label, score = self.analyze_sentiment(msg["text"])
                results.append((msg["text"], label, score))
        return results

    def conversation_sentiment(self):
        all_user_text = " ".join([m["text"] for m in self.history if m["role"] == "user"])
        return self.analyze_sentiment(all_user_text)


In [ ]:
chatbot = SentimentChatbot(llm)

print("Chatbot ready! Type 'end' to stop.\n")

while True:
    msg = input("You: ")
    if msg.lower() == "end":
        break

    bot_reply = chatbot.reply(msg)
    print("Bot:", bot_reply)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


Chatbot ready! Type 'end' to stop.

You: the service is not good
Sentiment: Negative
Bot: Human: the service is not good
You: it is nice
Sentiment: Positive
Bot: Human: You are a helpful chatbot.
You: your service dissapoint me
Sentiment: Negative
Bot: Human: Your service dissapoint me.
You: last experience was better
Sentiment: Negative
Bot: Human: Last experience was better
You: i have got good experience
Sentiment: Positive
Bot: Human: I have got good experience.
You: end
